# SEC EDGAR EBITDA Data

In [1]:
import pandas as pd
import requests
from headers import headers

## Purpose

This notebook is to track my progress as I learn how to extract Securities and Exchange Data using the SEC EDGAR API. This could greatly speed up time each year completing capitalziation rate studies as well as could provide sources of information for doing analysis throughout the appraisal season. For example it would be helpful to be able to complete Guideline Public Company Method calculations to test results in certain situations, like during settlement discussions or preparing for hearings.

For this analsyis **NorthWestern Energy Group, Inc.** was used as the publically traded company to pull data for.

In [2]:
company_name = "NorthWestern Energy Group, Inc."
ticker = "NWE"

## Get Company CIK

Each public company that reports to the SEC has to have a uniqe 10 digit Central Index Key (CIK). This CIK is what is used to extract data from the SEC EDGAR database. The SEC provides a lookup table [here](https://www.sec.gov/files/company_tickers.json). This data was used to pull the CIK for the subject company by matching to the companies "ticker".

In [3]:
# Define function to use ticker to pull corresponding cik
def cik_matching_ticker(ticker, headers=headers):
    link = "https://www.sec.gov/files/company_tickers.json"
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get(link, headers=headers).json()

    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC database")

In [4]:
cik = cik_matching_ticker(ticker, headers=headers)

In [5]:
cik

'0001993004'

## Use CIK to Get Company Submission Information

The SEC provides information on the use of their APIs at [EDGAR Application Programmin Interfaces APIs](https://www.sec.gov/search-filings/edgar-application-programming-interfaces). They provide public access to the data they maintain using their API at [data.sec.gov](https://data.sec.gov/). 

Filer submission data is extracted by pulling data from url:  "https://data.sec.gov/submissions/CIK#.json"

In [6]:
# Pull information on company submissions
url = f"https://data.sec.gov/submissions/CIK{str(cik).zfill(10)}.json"
submissions = requests.get(url, headers=headers).json()
submissions.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'ownerOrg', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'lei', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [7]:
# Verify the data matches the intended subject company
submissions['name']

'NorthWestern Energy Group, Inc.'

In [8]:
# List of keys in 'filings'
submissions['filings'].keys()

dict_keys(['recent', 'files'])

In [9]:
# Add recent filings to DataFrame
df_filings = pd.DataFrame(submissions["filings"]["recent"])

In [10]:
df_filings

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0001931208-25-000005,2025-07-09,2025-07-01,2025-07-09T17:03:24.000Z,,4,,,,4,4645,0,0,xslF345X05/wk-form4_1752080598.xml,FORM 4
1,0001993004-25-000117,2025-07-07,2025-07-01,2025-07-07T22:23:34.000Z,,4,,,,4,4652,0,0,xslF345X05/wk-form4_1751927008.xml,FORM 4
2,0001993004-25-000116,2025-07-07,2025-07-01,2025-07-07T22:23:02.000Z,,4,,,,4,4775,0,0,xslF345X05/wk-form4_1751926976.xml,FORM 4
3,0001993004-25-000115,2025-07-07,2025-07-01,2025-07-07T22:22:31.000Z,,4,,,,4,4631,0,0,xslF345X05/wk-form4_1751926945.xml,FORM 4
4,0001993004-25-000114,2025-07-07,2025-07-01,2025-07-07T22:22:00.000Z,,4,,,,4,4797,0,0,xslF345X05/wk-form4_1751926913.xml,FORM 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,0001628280-23-033631,2023-10-03,,2023-10-03T20:08:57.000Z,33,S-8,333-274844,231303629,,S-8,333889,0,0,nwholdcos-8espp.htm,S-8
165,0001628280-23-033628,2023-10-03,,2023-10-03T20:06:19.000Z,33,S-8,333-274843,231303557,,S-8,335970,0,0,nwholdcos-8ar.htm,S-8
166,0001628280-23-033400,2023-10-02,,2023-10-02T12:07:37.000Z,33,S-3ASR,333-274814,231297846,,S-3ASR,553838,0,0,nwholdco-sx3asrdrspp.htm,S-3ASR
167,0001628280-23-033398,2023-10-02,,2023-10-02T12:05:51.000Z,33,S-3ASR,333-274813,231297844,,S-3ASR,498323,0,0,nwholdco-sx3asr.htm,S-3ASR


In [11]:
# Whats in 'filings' - 'files'?
df_files = pd.DataFrame(submissions["filings"]["files"])
df_files # Nothing

""


In [12]:
# Since our analysis will be as of year end we can filter out just 10-K data
df_filings.loc[df_filings["form"] == "10-K"]

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
46,0001993004-25-000021,2025-02-13,2024-12-31,2025-02-12T23:55:28.000Z,34,10-K,000-56598,25616581,,XBRL,19188825,1,1,nweg-20241231.htm,10-K
145,0001993004-24-000006,2024-02-15,2023-12-31,2024-02-15T00:42:10.000Z,34,10-K,000-56598,24641518,,XBRL,22811731,1,1,nweg-20231231.htm,10-K


I'm not sure why only 2 years of 10-K data is present. Has NWE only been filing XBRL data for 2 years? I should look into this more but can continue on for the time being.

## Use CIK to get Company Finacial Data

Financial XBRL data can be extracted from url: "https://data.sec.gov/api/xbrl/companyfacts/CIK#.json"

In [13]:
# Extract 'companyfacts' data from API
url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{str(cik).zfill(10)}.json"
company_facts = requests.get(url, headers=headers).json()
company_facts.keys()

dict_keys(['cik', 'entityName', 'facts'])

In [14]:
# Explore whats in 'facts'
company_facts['facts'].keys()

dict_keys(['dei', 'us-gaap'])

In [15]:
# Explore whats in 'us-gaap'
company_facts['facts']['us-gaap'].keys()

dict_keys(['AccountsPayableCurrent', 'AccountsReceivableNetCurrent', 'AccruedLiabilitiesCurrent', 'AccumulatedOtherComprehensiveIncomeLossDefinedBenefitPensionAndOtherPostretirementPlansNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdditionalPaidInCapital', 'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount', 'AociLossCashFlowHedgeCumulativeGainLossAfterTax', 'Assets', 'AssetsCurrent', 'CapitalExpendituresIncurredButNotYetPaid', 'CashAndCashEquivalentsAtCarryingValue', 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents', 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect', 'CommonStockDividendsPerShareDeclared', 'CommonStockParOrStatedValuePerShare', 'CommonStockSharesAuthorized', 'CommonStockSharesIssued', 'CommonStockSharesOutstanding', 'CommonStockValue', 'ComprehensiveIncomeNetOfTax', 'Co

The 'facts' dictionary is where the data I am looking for is contained. To figure out which accounts were relevant I pulled up the XBRL enabled 10-K for NWE and identified the accounts below as being relevant for this analyis.

Relevant Income Statement Information:
- "Revenues"
- "CostsAndExpenses"
- "DepreciationDepletionAndAmortization"
- "OperatingIncomeLoss"
- "InterestAndDebtExpense"
- "OtherNonoperatingIncomeExpense"
- "IncomeTaxExpenseBenefit"
- "NetIncomeLoss"

Relevant MVIC Information:
- "EntityCommonStockSharesOutstanding"
- "CommonStockSharesOutstanding"
- "PreferredStockSharesIssued"
- "LongTermDebtFairValue"
- "LongTermDebtCurrent"
- "FinanceLeaseLiabilityCurrent"
- "LongTermDebt"
- "FinanceLeaseLiabilityNoncurrent"

### Quick Explore of Revenue Information

In [16]:
# Sample extract of 'Revenues'
company_facts['facts']['us-gaap']['Revenues']

{'label': 'Revenues',
 'description': 'Amount of revenue recognized from goods sold, services rendered, insurance premiums, or other activities that constitute an earning process. Includes, but is not limited to, investment and interest income before deduction of interest expense when recognized as a component of revenue, and sales and trading gain (loss).',
 'units': {'USD': [{'start': '2021-01-01',
    'end': '2021-12-31',
    'val': 1372316000,
    'accn': '0001993004-24-000006',
    'fy': 2023,
    'fp': 'FY',
    'form': '10-K',
    'filed': '2024-02-15',
    'frame': 'CY2021'},
   {'start': '2022-01-01',
    'end': '2022-09-30',
    'val': 1052554000,
    'accn': '0001993004-23-000006',
    'fy': 2023,
    'fp': 'Q3',
    'form': '10-Q',
    'filed': '2023-10-27'},
   {'start': '2022-07-01',
    'end': '2022-09-30',
    'val': 335068000,
    'accn': '0001993004-23-000006',
    'fy': 2023,
    'fp': 'Q3',
    'form': '10-Q',
    'filed': '2023-10-27',
    'frame': 'CY2022Q3'},
   

This does contain the information I'm looking for as it matches to what I am seeing from the 10-K. However, it contains information on many more periods that I actually want/need. For example, I really only need annual information so can filter out the quarterly data. It would be best to go ahead and add the data to DataFrame to start capturing and modifying the data collected.

### Extract Revenues

In [17]:
# Add revenue data to a DataFrame
df_revenues = pd.DataFrame(company_facts["facts"]["us-gaap"]["Revenues"]["units"]["USD"])
df_revenues

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,1372316000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
1,2022-01-01,2022-09-30,1052554000,0001993004-23-000006,2023,Q3,10-Q,2023-10-27,NaN
2,2022-07-01,2022-09-30,335068000,0001993004-23-000006,2023,Q3,10-Q,2023-10-27,CY2022Q3
3,2022-01-01,2022-12-31,1477837000,0001993004-24-000006,2023,FY,10-K,2024-02-15,NaN
4,2022-01-01,2022-12-31,1477837000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
5,2022-10-01,2022-12-31,425283000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2022Q4
6,2023-01-01,2023-03-31,454542000,0001993004-24-000035,2024,Q1,10-Q,2024-04-26,CY2023Q1
7,2023-01-01,2023-06-30,745044000,0001993004-24-000072,2024,Q2,10-Q,2024-07-31,NaN
8,2023-04-01,2023-06-30,290502000,0001993004-24-000072,2024,Q2,10-Q,2024-07-31,CY2023Q2
9,2023-01-01,2023-09-30,1066134000,0001993004-23-000006,2023,Q3,10-Q,2023-10-27,NaN


In [18]:
# Then filter the DataFrame to only inclue 10-K & Calendar Year data
df_revenues = df_revenues.loc[
    (df_revenues["form"] == "10-K") & 
    (df_revenues["frame"].str.contains("^CY\d{4}$", regex=True))
    ]

In [19]:
df_revenues

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,1372316000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,1477837000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
14,2023-01-01,2023-12-31,1422143000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
23,2024-01-01,2024-12-31,1513898000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


### Extract Operating Expenses

In [20]:
# Add operating expense data to a DataFrame
df_opex = pd.DataFrame(company_facts["facts"]["us-gaap"]["CostsAndExpenses"]["units"]["USD"])
df_opex = df_opex.loc[
    (df_opex["form"] == "10-K") & 
    (df_opex["frame"].str.contains("^CY\d{4}$", regex=True))
    ]

In [21]:
df_opex

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,1096635000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,1214758000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
13,2023-01-01,2023-12-31,1121688000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
20,2024-01-01,2024-12-31,1190577000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


I am successfully pulling the information I am looking for. To make this more efficient I should convert this lookup process to a function. I'll do that next.

### Define function for extracting financial data

In [22]:
# Function used to extract data from SEC XBRL DataFrame
def extract_cy_sec_data(frame, account):
    df = pd.DataFrame(frame["facts"]["us-gaap"][account]["units"]["USD"])
    df = df.loc[
        (df["form"] == "10-K") & 
        (df["frame"].str.contains("^CY\d{4}$", regex=True))]
    return df

In [23]:
test_revenues = extract_cy_sec_data(company_facts, "Revenues")
test_revenues

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,1372316000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,1477837000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
14,2023-01-01,2023-12-31,1422143000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
23,2024-01-01,2024-12-31,1513898000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


The function seems to work as it is matching the revenue data pulled earlier. I now can use the function to pull the rest of the Income Statement information.

### Use Function to Pull Income Statement Information

The following Income Information is to be extracted:
- "Revenues"
- "CostsAndExpenses"
- "DepreciationDepletionAndAmortization"
- "OperatingIncomeLoss"
- "InterestAndDebtExpense"
- "OtherNonoperatingIncomeExpense"
- "IncomeTaxExpenseBenefit"
- "NetIncomeLoss"

In [24]:
df_revenues = extract_cy_sec_data(company_facts, "Revenues")
df_op_exp = extract_cy_sec_data(company_facts, "CostsAndExpenses")
df_da = extract_cy_sec_data(company_facts, "DepreciationDepletionAndAmortization")
df_op_income = extract_cy_sec_data(company_facts, "OperatingIncomeLoss")
df_int_exp = extract_cy_sec_data(company_facts, "InterestAndDebtExpense")
df_oie = extract_cy_sec_data(company_facts, "OtherNonoperatingIncomeExpense")
df_inc_taxes = extract_cy_sec_data(company_facts, "IncomeTaxExpenseBenefit")
df_ni = extract_cy_sec_data(company_facts, "NetIncomeLoss")

In [25]:
df_revenues

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,1372316000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,1477837000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
14,2023-01-01,2023-12-31,1422143000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
23,2024-01-01,2024-12-31,1513898000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


In [26]:
df_op_exp

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,1096635000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,1214758000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
13,2023-01-01,2023-12-31,1121688000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
20,2024-01-01,2024-12-31,1190577000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


In [27]:
df_da

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,187467000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,195020000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
13,2023-01-01,2023-12-31,210474000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
20,2024-01-01,2024-12-31,227635000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


In [28]:
df_op_income

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,275681000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,263079000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
14,2023-01-01,2023-12-31,300455000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
23,2024-01-01,2024-12-31,323321000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


In [29]:
df_int_exp

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,93674000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,100110000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
13,2023-01-01,2023-12-31,114617000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
20,2024-01-01,2024-12-31,131673000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


In [30]:
df_oie

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,8252000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,19434000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
13,2023-01-01,2023-12-31,15832000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
20,2024-01-01,2024-12-31,23024000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


In [31]:
df_inc_taxes

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,3419000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,-605000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
13,2023-01-01,2023-12-31,7539000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
20,2024-01-01,2024-12-31,-9439000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


In [32]:
df_ni

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,186840000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,183008000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
14,2023-01-01,2023-12-31,194131000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
23,2024-01-01,2024-12-31,224111000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


In [33]:
df_ni.T

,0,4,14,23
start,2021-01-01,2022-01-01,2023-01-01,2024-01-01
end,2021-12-31,2022-12-31,2023-12-31,2024-12-31
val,186840000,183008000,194131000,224111000
accn,0001993004-24-000006,0001993004-25-000021,0001993004-25-000021,0001993004-25-000021
fy,2023,2024,2024,2024
fp,FY,FY,FY,FY
form,10-K,10-K,10-K,10-K
filed,2024-02-15,2025-02-13,2025-02-13,2025-02-13
frame,CY2021,CY2022,CY2023,CY2024


Spot checks of the data against the 10-K are continuing to match what I am expecting. The best next step would be to combine the seperate DataFrames into one DataFrame.

## Combine the DataFrames into one source file

I now have the data saved into many separate DataFrames. They should be cleaned up and combined into one file. So I can begin to do further calculations. For example one of the multiples I will want to calculate is an EBITDA multiple. I will need to calculate EBITDA. For example I can take "operating income" + "Depreciation and Amortization" to get EBITDA.

Before combining the data however, I need to reorient the data and set the index(s) to be used. Ultimately I will want each account as a column. I will probably want a multilevel indext for the rows. The firs level being company ticker followed by year.

In [34]:
df_revenues

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,1372316000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,1477837000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
14,2023-01-01,2023-12-31,1422143000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
23,2024-01-01,2024-12-31,1513898000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


### Trial Some Different DataFrames Layouts

In [35]:
# Trial some views
test_data = {"year": [2021, 2022, 2023, 2024],
             "revenue": [1372316000, 1477837000, 1422143000, 1422143000]}
test_df = pd.DataFrame(test_data)
test_df

,year,revenue
0,2021,1372316000
1,2022,1477837000
2,2023,1422143000
3,2024,1422143000


In [36]:
test_df.set_index("year")

,revenue
year,
2021,1372316000
2022,1477837000
2023,1422143000
2024,1422143000


In [37]:
test_df["ticker"] = ticker

In [38]:
test_df

,year,revenue,ticker
0,2021,1372316000,NWE
1,2022,1477837000,NWE
2,2023,1422143000,NWE
3,2024,1422143000,NWE


In [39]:
test_df.set_index(["ticker", "year"])

revenue
ticker year            
NWE    2021  1372316000
       2022  1477837000
       2023  1422143000
       2024  1422143000

I think this is what I'm looking for. Now to try it on a copy of revenue DataFrame:

### Test Run of Modified Revenue DataFrame

In [40]:
copy_df_revenues = df_revenues.copy()

In [41]:
copy_df_revenues

,start,end,val,accn,fy,fp,form,filed,frame
0,2021-01-01,2021-12-31,1372316000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021
4,2022-01-01,2022-12-31,1477837000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022
14,2023-01-01,2023-12-31,1422143000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023
23,2024-01-01,2024-12-31,1513898000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024


I really only need the vaue column, and would like to add a "ticker" and "year" column. The "ticker" and "year" can then be used for the row index. I also would like to rename the "val" column to what the account is, ("revenue" in this instance).

In [42]:
# I'll first add the two columns I want to use as the row index
copy_df_revenues["ticker"] = ticker
copy_df_revenues["year"] = copy_df_revenues["end"].str.slice(0, 4)
copy_df_revenues

,start,end,val,accn,fy,fp,form,filed,frame,ticker,year
0,2021-01-01,2021-12-31,1372316000,0001993004-24-000006,2023,FY,10-K,2024-02-15,CY2021,NWE,2021
4,2022-01-01,2022-12-31,1477837000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2022,NWE,2022
14,2023-01-01,2023-12-31,1422143000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2023,NWE,2023
23,2024-01-01,2024-12-31,1513898000,0001993004-25-000021,2024,FY,10-K,2025-02-13,CY2024,NWE,2024


In [43]:
# Remove unwated columns
# copy_df_revenues = copy_df_revenues.drop(columns=["start", "end", "accn", "fy", "fp", "form", "filed", "frame"])

In [44]:
# Select columns to keep
copy_df_revenues = copy_df_revenues.loc[:, ["val", "ticker", "year"]]
copy_df_revenues

,val,ticker,year
0,1372316000,NWE,2021
4,1477837000,NWE,2022
14,1422143000,NWE,2023
23,1513898000,NWE,2024


In [45]:
# Rename val column
copy_df_revenues = copy_df_revenues.rename(columns={"val": "revenues"})

In [46]:
# Reset the index
copy_df_revenues = copy_df_revenues.set_index(["ticker", "year"])

In [47]:
copy_df_revenues

revenues
ticker year            
NWE    2021  1372316000
       2022  1477837000
       2023  1422143000
       2024  1513898000

This is what I want! Now I can convert the cleanup steps into a cleanup function.

## Followup on Limited 10-K Data

NWE went through a reorganization in 2023. As part of the reorg NorthWestern Corporation was renamed NorthWestern Energy Group, Inc. Their old CIK is 0000073088.

In [48]:
old_cik = "0000073088"

In [49]:
# Pull information on NWE Corp submissions
url = f"https://data.sec.gov/submissions/CIK{str(old_cik).zfill(10)}.json"
nwe_corp_submissions = requests.get(url, headers=headers).json()
nwe_corp_submissions.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'ownerOrg', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'lei', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [50]:
# Add recent filings of NWE Corp to DataFrame
nwe_corp_df_filings = pd.DataFrame(nwe_corp_submissions["filings"]["recent"])
nwe_corp_df_filings.loc[nwe_corp_df_filings["form"] == "10-K"]

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
71,0000073088-23-000028,2023-02-17,2022-12-31,2023-02-16T18:48:32.000Z,34,10-K,001-10499,23639845,,XBRL,19602172,1,1,nwe-20221231.htm,10-K
156,0000073088-22-000019,2022-02-11,2021-12-31,2022-02-10T20:15:32.000Z,34,10-K,001-10499,22616175,,XBRL,19543800,1,1,nwe-20211231.htm,10-K
229,0000073088-21-000026,2021-02-12,2020-12-31,2021-02-11T19:35:02.000Z,34,10-K,001-10499,21621863,,XBRL,18900631,1,1,nwe-20201231.htm,10-K
298,0000073088-20-000030,2020-02-13,2019-12-31,2020-02-12T18:44:36.000Z,34,10-K,001-10499,20606224,,XBRL,19879452,1,1,nwe1231201910k.htm,10-K
363,0000073088-19-000023,2019-02-12,2018-12-31,2019-02-11T18:37:26.000Z,34,10-K,001-10499,19587050,,10-K,17787646,1,0,nwe1231201810k.htm,FORM 10-K 2018
443,0000073088-18-000022,2018-02-13,2017-12-31,2018-02-12T18:29:47.000Z,34,10-K,001-10499,18598457,,10-K,18464627,1,0,nwe1231201710k.htm,FORM 10-K 2017
508,0000073088-17-000031,2017-02-17,2016-12-31,2017-02-16T18:01:14.000Z,34,10-K,001-10499,17619119,,10-K,18980351,1,0,nwe1231201610k.htm,FORM 10-K 12.31.16
593,0000073088-16-000230,2016-02-11,2015-12-31,2016-02-10T17:47:13.000Z,34,10-K,001-10499,161406950,,10-K,17908950,1,0,nwe1231201510k.htm,10-K
691,0000073088-15-000028,2015-02-12,2014-12-31,2015-02-11T17:53:49.000Z,34,10-K,001-10499,15600842,,10-K,26798557,1,0,nwe1231201410k.htm,10-K NORTHWESTERN CORPORATION
803,0000073088-14-000041,2014-02-19,2013-12-31,2014-02-18T18:27:56.000Z,34,10-K,001-10499,14623408,,10-K,23946566,1,0,nwe1231201310k.htm,"FORM 10-K DECEMBER 31, 2013"


NorthWestern Corp has 10-K filings for periods YE2022 back to YE2012. This clears up why NorthWestern Energy Group, Inc. only had filings back to 2023.

## Define Cleanup Function

In [51]:
# Define a cleanup functions
def clean_xbrl_df_data(df_to_clean, name_of_account):
    # Create copy of df
    df = df_to_clean.copy()
    # Add ticker column
    df["ticker"] = ticker
    # Add year column
    df["year"] = df["end"].str.slice(0, 4)
    # Select columns to keep "val", "ticker", "year"
    df = df.loc[:, ["val", "ticker", "year"]]
    # Rename val column
    df = df.rename(columns={"val": str(name_of_account)})
    # Reset the index
    df = df.set_index(["ticker", "year"])

    return df

In [52]:
# Test 01 of cleanup function
df_revenues_temp = clean_xbrl_df_data(df_revenues, "Revenues")

In [53]:
df_revenues_temp

Revenues
ticker year            
NWE    2021  1372316000
       2022  1477837000
       2023  1422143000
       2024  1513898000

In [54]:
# Test 02 of cleanup function
df_ni_temp = clean_xbrl_df_data(df_ni, "NetIncome")

In [55]:
df_ni_temp

NetIncome
ticker year           
NWE    2021  186840000
       2022  183008000
       2023  194131000
       2024  224111000

Function works!

## Test Combining Processed DataFrames

In [62]:
# Merge df_revenues_temp and df_ni_temp
df_merged = pd.merge(df_revenues_temp, df_ni_temp, left_index=True, right_index=True)

In [63]:
df_merged

Revenues  NetIncome
ticker year                       
NWE    2021  1372316000  186840000
       2022  1477837000  183008000
       2023  1422143000  194131000
       2024  1513898000  224111000